In [1]:
# Standard Library Packages
import subprocess
from io import BytesIO

# Other Packages
import folium
import holoviews as hv
import hvplot.pandas
import pandas as pd
import requests
import warnings


# Ignore warnings
warnings.filterwarnings("ignore")

# Streamflow in the Connecticut River at Thompsonville, Connecticut

### Site Description
* This analysis focuses on streamflow in the Connecticut River in Thompsonville, CT.
* The Connecticut River is the longest river in New England. The Connecticut River watershed includes five states (Connecticut, Massachusetts, New Hampshire, Vermont, Maine) as well as one Canadian province.

<img src="https://upload.wikimedia.org/wikipedia/commons/3/39/Connecticut_River_Map.png" style="float:left;width:300px">
<img src="https://upload.wikimedia.org/wikipedia/commons/7/7e/Connecticut_River_From_Gillette_Castle.jpg" style="float:left; height: 400px">

In [2]:
# Define Latitute and Longitude
sg_lat = 41.9873186
sg_lon = -72.6053669

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], popup="Connecticut River at Thompsonville, CT").add_to(m)

# Display the map
m

In [3]:
# Get Omaha Data (using ChatGPT suggested code)
ct_Data_url = ("https://waterdata.usgs.gov/nwis/dv?cb_00060=on&format="
               "rdb&site_no=01184000&legacy=&referred_module=sw&period="
               "&begin_date=1928-10-01&end_date=2023-09-17")

# Send an HTTP GET request to the URL
ct_response = requests.get(ct_Data_url)

# Check if the request was successful (status code 200)
if ct_response.status_code == 200:
    # Print the content of the response (HTML, JSON, etc.)
    print(ct_response.text)
else:
    print(f"Request failed with status code {ct_response.status_code}")

# ---------------------------------- WARNING ----------------------------------------
# Some of the data that you have obtained from this U.S. Geological Survey database
# may not have received Director's approval. Any such data values are qualified
# as provisional and are subject to revision. Provisional data are released on the
# condition that neither the USGS nor the United States Government may be held liable
# for any damages resulting from its use.
#
# Additional info: https://help.waterdata.usgs.gov/policies/provisional-data-statement
#
# File-format description:  https://help.waterdata.usgs.gov/faq/about-tab-delimited-output
# Automated-retrieval info: https://help.waterdata.usgs.gov/faq/automated-retrievals
#
# Contact:   gs-w_waterdata_support@usgs.gov
# retrieved: 2023-09-21 11:20:48 EDT       (vaww01)
#
# Data for the following 1 site(s) are contained in this file
#    USGS 01184000 CONNECTICUT RIVER AT THOMPSONVILLE, CT
# -------------------------------------------------

In [4]:
ct_df = pd.read_csv(
    BytesIO(ct_response.content),
    comment='#',
    delimiter='\t',
    skiprows=30,
    names=["Agency", "Site_Number", "Date", "Streamflow_cfs", "cd"],
    index_col='Date',
    parse_dates=True,
)

In [5]:
# Resampling from daily to annual
ct_annual_df = ct_df[['Streamflow_cfs']].resample('AS').max()

ct_annual_plt = ct_annual_df.hvplot(y='Streamflow_cfs').opts(
    title="Streamflow for the Connecticut River in Thompsonville, CT (1928 to 2023)",
    xlabel="Year",
    ylabel="Streamflow (ft^3/s)"
)

ct_annual_plt


:Curve   [Date]   (Streamflow_cfs)

In [6]:
# Data from the period after dams were built.
ct_annual_df_recent = ct_annual_df["1971-01-01":"2023-01-01"].copy()

# Plot of Annual Streamflow
ct_post1950_plt = ct_annual_df_recent.hvplot(y='Streamflow_cfs').opts(
        title="Streamflow for the Connecticut River in Thompsonville, CT (1971 to 2023)",
    xlabel="Year",
    ylabel="Streamflow (ft^3/s)"
)

ct_post1950_plt

:Curve   [Date]   (Streamflow_cfs)

In [7]:
# Add exeedence probabilities
ct_annual_df_recent['exceed_prob'] = (
    ct_annual_df_recent
    .rank(ascending=False)
    / len(ct_annual_df_recent)
)

# Add Return Rate
ct_annual_df_recent['return_rate'] = (
    1 / ct_annual_df_recent['exceed_prob']
)

In [8]:
# Largest Floods
high_return_rates = ct_annual_df_recent[ct_annual_df_recent['return_rate'] > 10]


### Summary
* The flooding in 2023 that has adversely affected farmers in Connecticut is the largest flood in a dozen years. However, it is not a particularly unusual event (return rate of 8.2)
* There were two major floods in 1936 and 1938. The US Army Corps of Engineers subsequently built 16 flood-control dams in the Connecticut River between 1940 and 1971. Streamflow has never surpassed 200,000 cubic feet per second at this location since that time.
* There were two additional large floods in 1984 and 1987.

In [9]:
%%capture
%%bash
jupyter nbconvert streamflow_time_series.ipynb --to html --no-input